In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
!pip install pytorch-transformers
!pip install transformers
!pip install nltk


     |████████████████████████████████| 184kB 17.9MB/s 
     |████████████████████████████████| 1.2MB 41.7MB/s 
     |████████████████████████████████| 890kB 48.9MB/s 
     |████████████████████████████████| 133kB 52.4MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 7.3MB 48.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f16453ca1fd717c45849c49117a9659a6d93b3856517f37b8ffb02c9f25d9a21
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.20.27 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 1.9MB 17.8MB/s 
     |████████████████████████████████| 3.2MB 53.0MB/s 


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
!pip install tweet-preprocessor

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import DebertaTokenizer, DebertaModel, DebertaForSequenceClassification
from transformers import AdamW
import nltk
from nltk.stem import 	WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# % matplotlib inline


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla T4'

In [7]:
MAX_LEN = 128

In [14]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Covid_fake_news/COVID Fake News Data.csv')

In [31]:
train = train.rename(columns={ 'headlines' : 'text'})
train = train.rename(columns={ 'outcome' : 'label'})
train['id'] = range(1, len(train) + 1)

In [16]:
df, val_df, test_df = np.split(train.sample(frac=1), [int(.6*len(train)), int(.8*len(train))])

df = df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


print("Size of training set: {}".format(len(df)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 6120
Size of validation set: 2040
Size of testing set: 2041


In [17]:
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer  = PorterStemmer()


In [18]:
p.set_options(p.OPT.URL, p.OPT.EMOJI)

def preprocess(row, lemmatizer, stemmer):
  text = row['text']
  # text = text.strip('\xa0')
  text = p.clean(text)
  tokenization = nltk.word_tokenize(text)     
  tokenization = [w for w in tokenization if not w in stop_words]
#   text = ' '.join([porter_stemmer.stem(w) for w in tokenization])
#   text = ' '.join([lemmatizer.lemmatize(w) for w in tokenization])
  # text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return text


In [19]:
df['text'] = df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)
val_df['text'] = val_df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)
test_df['text'] = test_df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)


In [20]:
def map_label(row):
  return 0 if row['label']==1 else 1

df['label_encoded'] = df.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [21]:
train_sentences = df.text.values
val_sentences = val_df.text.values
test_sentences = test_df.text.values

train_labels = df.label_encoded.values
val_labels = val_df.label_encoded.values


In [ ]:
# tokenizer_sentences = np.array(list(df.tweet.values) + list(val_df.tweet.values))

In [22]:
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base', do_lower_case=True)


In [23]:
def Encode_TextWithAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=True):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids'],encoded_dict['attention_mask']

def Encode_TextWithoutAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=False):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids']

def get_TokenizedTextWithAttentionMask(sentenceList, tokenizer):
    token_ids_list,attention_mask_list = [],[]
    for sentence in sentenceList:
        token_ids,attention_mask = Encode_TextWithAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
        attention_mask_list.append(attention_mask)
    return token_ids_list,attention_mask_list

def get_TokenizedText(sentenceList, tokenizer):
    token_ids_list = []
    for sentence in sentenceList:
        token_ids = Encode_TextWithoutAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
    return token_ids_list

In [24]:
train_token_ids,train_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(train_sentences,tokenizer))
val_token_ids,val_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(val_sentences,tokenizer))
test_token_ids,test_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(test_sentences,tokenizer))

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

In [25]:
batch_size = 32

train_data = TensorDataset(train_token_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(val_token_ids, val_attention_masks, val_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_token_ids, test_attention_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [26]:
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2).cuda()

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base

In [27]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [28]:
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [29]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [30]:
train_loss_set = []
best_val_accuracy = 0.50
directory_path =  '/content/gdrive/MyDrive/4th_sem/dataset/Covid_fake_news'
epochs = 15

for _ in trange(epochs, desc="Epoch"):
  
  model.train()
  
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    optimizer.zero_grad()
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    loss.backward()
    optimizer.step()
        
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
  model.eval()

  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = output[0]
    
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  Validation_Accuracy = (eval_accuracy/nb_eval_steps)
  if(Validation_Accuracy >= best_val_accuracy):
    torch.save(model.state_dict(), directory_path+'/models/DeBERTa_base_best_model.ckpt')
    best_val_accuracy = Validation_Accuracy
    print('Model Saved')

    


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Train loss: 0.1363609112716707
Validation Accuracy: 0.9697265625


Epoch:   7%|▋         | 1/15 [03:04<42:57, 184.09s/it]

Model Saved
Train loss: 0.058326424843471614


Epoch:  13%|█▎        | 2/15 [06:07<39:51, 183.97s/it]

...

Validation Accuracy: 0.97802734375
Train loss: 0.005804724321687142


Epoch:  93%|█████████▎| 14/15 [42:57<03:03, 183.44s/it]

Validation Accuracy: 0.98046875
Train loss: 0.0004588740438104348
Validation Accuracy: 0.9814453125


Epoch: 100%|██████████| 15/15 [46:02<00:00, 184.17s/it]

Model Saved


In [32]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import XLNetTokenizer, XLNetForSequenceClassification,BertTokenizer, BertForSequenceClassification, DebertaTokenizer, DebertaForSequenceClassification, RobertaTokenizerFast, RobertaForSequenceClassification, ElectraTokenizer, ElectraForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline


In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla T4'

In [34]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Covid_fake_news/COVID Fake News Data.csv')

In [35]:
train = train.rename(columns={ 'headlines' : 'text'})
train = train.rename(columns={ 'outcome' : 'label'})
train['id'] = range(1, len(train) + 1)

In [36]:
train.head()

,text,label,id
0,A post claims compulsory vacination violates t...,0,1
1,A photo claims that this person is a doctor wh...,0,2
2,Post about a video claims that it is a protest...,0,3
3,All deaths by respiratory failure and pneumoni...,0,4
4,The dean of the College of Biologists of Euska...,0,5


In [37]:
df, val_df, test_df = np.split(train.sample(frac=1), [int(.6*len(train)), int(.8*len(train))])

df = df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
test_label_df = test_df

print("Size of training set: {}".format(len(df)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 6120
Size of validation set: 2040
Size of testing set: 2041


In [38]:
def preprocess(row):
  text = row['text']
  # text = text.strip('\xa0')
  text = p.clean(text)
  # text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return text


In [39]:
def map_label(row):
  return 0 if row['label']==1 else 1

df['label_encoded'] = df.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [40]:
train_sentences = df.text.values
train_token_ids = df.id.values
val_sentences = val_df.text.values
val_token_ids = val_df.id.values
test_sentences = test_df.text.values
test_token_ids = test_df.id.values


In [41]:
train_sentences = [sentence + " [SEP] [CLS]" for sentence in train_sentences]
train_labels = df.label_encoded.values
val_sentences = [sentence + " [SEP] [CLS]" for sentence in val_sentences]
val_labels = val_df.label_encoded.values
test_sentences = [sentence + " [SEP] [CLS]" for sentence in test_sentences]


In [42]:
MAX_LEN = 128
batch_size = 64


In [43]:
def get_dataloader(network, train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids):
  dic = {"XLNET": "xlnet-base-cased", "BERT": "bert-base-uncased", "ROBERT": "roberta-base", "XLM-ROBERT": "xlm-roberta-base", "DeBERTa": "microsoft/deberta-base", "Electra": "google/electra-base-discriminator"}
  if network=='XLNET':
    tokenizer = XLNetTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='DeBERTa':
    tokenizer = DebertaTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='BERT':
    tokenizer = BertTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='Electra':
    tokenizer = ElectraTokenizer.from_pretrained(dic[network], do_lower_case=True)
  else:
    tokenizer = RobertaTokenizerFast.from_pretrained(dic[network], do_lower_case=True)

  tokenized_train_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
  print ("Tokenize the first sentence:")
  print (tokenized_train_texts[0])

  tokenized_val_texts = [tokenizer.tokenize(sent) for sent in val_sentences]
  tokenized_test_texts = [tokenizer.tokenize(sent) for sent in test_sentences]
  input_train_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_train_texts]
  input_val_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_val_texts]
  input_test_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_test_texts]

  input_train_ids = pad_sequences(input_train_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_val_ids = pad_sequences(input_val_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_test_ids = pad_sequences(input_test_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  # Create attention masks
  train_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_train_ids:
    seq_mask = [float(i>0) for i in seq]
    train_attention_masks.append(seq_mask)
  train_masks = np.array(train_attention_masks)

  # Create attention masks
  val_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_val_ids:
    seq_mask = [float(i>0) for i in seq]
    val_attention_masks.append(seq_mask)
  validation_masks = np.array(val_attention_masks)

  # Create attention masks
  test_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_test_ids:
    seq_mask = [float(i>0) for i in seq]
    test_attention_masks.append(seq_mask)
  test_masks = np.array(test_attention_masks)

  train_inputs = torch.tensor(input_train_ids)
  validation_inputs = torch.tensor(input_val_ids)
  test_inputs = torch.tensor(input_test_ids)
  train_labels = torch.tensor(train_labels)
  validation_labels = torch.tensor(val_labels)
  train_masks = torch.tensor(train_masks)
  validation_masks = torch.tensor(validation_masks)
  test_masks = torch.tensor(test_masks)

  train_data = TensorDataset(torch.tensor(train_token_ids), train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  validation_data = TensorDataset(torch.tensor(val_token_ids), validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  test_data = TensorDataset(torch.tensor(test_token_ids), test_inputs, test_masks)
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

  return train_dataloader, validation_dataloader, test_dataloader


In [44]:
train_dataloader_DeBERTa, val_dataloader_DeBERTa, test_dataloader_DeBERTa = get_dataloader("DeBERTa", train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids)

Tokenize the first sentence:
['272', '2708', '1139', '326', '275', '799', '391', '468', '6184', '95', '26391', '129', '241', '31216', '276', '503', '22940', '26391', '126', '251', '286', '257', '642', '70', '2775', '351', '442', '3762', '1664', '289', '84', '38247', '780', '1332', '19183', '547', '1043', '284', '307', '23543', '351', '26920', '615', '19397', '13', '[SEP]', '[CLS]']


In [45]:
model4 = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)
directory_path = '/content/gdrive/MyDrive/4th_sem/dataset/Covid_fake_news/models'
model4.load_state_dict(torch.load(directory_path+'/DeBERTa_base_best_model.ckpt'))
model4.eval()
model4.cuda()



Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermed

In [46]:
def get_model_preds_labels(model, dataloader, mode='train'):
  data_vectors = []
  labels = []
  ids = []

  with torch.no_grad():
      correct = 0
      total = 0
      for i, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        if mode=='test':
            token_ids, b_input_ids, b_input_mask = batch
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            for point, tk_id in zip(F.softmax(outputs.logits).tolist(), token_ids.tolist()):
              data_vectors.append(point)       
              ids.append(tk_id)     
        else:
          # Unpack the inputs from our dataloader
          token_ids, b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          for point, lab, tk_id in zip(F.softmax(outputs.logits).tolist(), b_labels.tolist(), token_ids.tolist()):
            data_vectors.append(point)
            labels.append(lab)
            ids.append(tk_id)

  if mode=='test':
    return data_vectors, ids
  else:
    return data_vectors, labels, ids


#Train Data

In [47]:
m4, _, train_ids4 = get_model_preds_labels(model4, train_dataloader_DeBERTa)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [48]:
pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in m4]
train_pred_df = pd.DataFrame({'id': train_ids4, 'predicted_label': pred_labels})
train_pred_df = df[["id", "label"]].merge(train_pred_df, on='id', how='left')
train_pred_df.head()

,id,label,predicted_label
0,5501,0,0
1,6801,0,0
2,4118,0,0
3,6811,0,0
4,8048,0,0


In [49]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score


In [50]:
confusion_matrix(train_pred_df['label'].values, train_pred_df['predicted_label'].values)

array([[5805,   35],
       [  35,  245]])

In [51]:
print(accuracy_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values))
print(precision_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))
print(recall_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))
print(f1_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))


0.988562091503268
0.988562091503268
0.988562091503268
0.988562091503268


#Validation Data

In [52]:
v4, _, v_ids4 = get_model_preds_labels(model4, val_dataloader_DeBERTa)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [53]:
val_pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in v4]
val_pred_df = pd.DataFrame({'id': v_ids4, 'predicted_label': val_pred_labels})
val_pred_df = val_df[["id", "label"]].merge(val_pred_df, on='id', how='left')
val_pred_df.head()



,id,label,predicted_label
0,7452,0,0
1,8866,0,0
2,1338,0,0
3,7753,0,0
4,3791,0,0


In [54]:
confusion_matrix(val_pred_df['label'].values, val_pred_df['predicted_label'].values)

array([[1922,   14],
       [  15,   89]])

In [55]:
print(accuracy_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values))
print(precision_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))
print(recall_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))
print(f1_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))

0.9857843137254902
0.9857843137254902
0.9857843137254902
0.9857843137254902


#Test Data

In [56]:
t4, t_ids4 = get_model_preds_labels(model4, test_dataloader_DeBERTa, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [57]:
test_pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in t4]
test_pred_df = pd.DataFrame({'id': t_ids4, 'predicted_label': test_pred_labels})
test_pred_df = test_label_df[["id", "label"]].merge(test_pred_df, on='id', how='left')
test_pred_df.head()

,id,label,predicted_label
0,661,0,0
1,2182,0,0
2,4695,0,0
3,5776,0,0
4,6571,0,0


In [58]:
confusion_matrix(test_pred_df['label'].values, test_pred_df['predicted_label'].values)

array([[1935,   16],
       [   9,   81]])

In [59]:
print(accuracy_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values))
print(precision_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro'))
print(recall_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro'))
print(f1_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro'))

0.9877511024007839
0.9877511024007839
0.9877511024007839
0.9877511024007839
